<a href="https://colab.research.google.com/github/Tuhinm2002/kaggle_competitions/blob/main/classification/binary_prediction_of_poisonous_mushrooms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e8/sample_submission.csv
/kaggle/input/playground-series-s4e8/train.csv
/kaggle/input/playground-series-s4e8/test.csv


In [ ]:
sample_data = pd.read_csv("/kaggle/input/playground-series-s4e8/sample_submission.csv")
train_data = pd.read_csv("/kaggle/input/playground-series-s4e8/train.csv")
test_data = pd.read_csv("/kaggle/input/playground-series-s4e8/test.csv")

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.optimizers import SGD,AdamW
from tensorflow.keras.layers import Layer,Dense,BatchNormalization,Input,Flatten
from tensorflow.keras.losses import Loss,MSE

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef

2024-08-14 17:16:59.271883: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-14 17:16:59.272045: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-14 17:16:59.435740: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
train_data.head()

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,1,p,4.51,x,h,o,f,a,c,n,...,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,2,e,6.94,f,s,b,f,x,c,w,...,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3,e,3.88,f,y,g,f,s,NaN,g,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,4,e,5.85,x,l,w,f,d,NaN,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a


In [ ]:
test_data.head()

,id,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,3116945,8.64,x,NaN,n,t,NaN,NaN,w,11.13,...,b,NaN,w,u,w,t,g,NaN,d,a
1,3116946,6.90,o,t,o,f,NaN,c,y,1.27,...,NaN,NaN,n,NaN,NaN,f,f,NaN,d,a
2,3116947,2.00,b,g,n,f,NaN,c,n,6.18,...,NaN,NaN,n,NaN,NaN,f,f,NaN,d,s
3,3116948,3.47,x,t,n,f,s,c,n,4.98,...,NaN,NaN,w,NaN,n,t,z,NaN,d,u
4,3116949,6.17,x,h,y,f,p,NaN,y,6.73,...,NaN,NaN,y,NaN,y,t,NaN,NaN,d,u


In [ ]:
train_data.isna().sum()

id                            0
class                         0
cap-diameter                  4
cap-shape                    40
cap-surface              671023
cap-color                    12
does-bruise-or-bleed          8
gill-attachment          523936
gill-spacing            1258435
gill-color                   57
stem-height                   0
stem-width                    0
stem-root               2757023
stem-surface            1980861
stem-color                   38
veil-type               2957493
veil-color              2740947
has-ring                     24
ring-type                128880
spore-print-color       2849682
habitat                      45
season                        0
dtype: int64

In [ ]:
len(train_data) - 128880

2988065

In [ ]:
le = LabelEncoder()

In [ ]:
le.fit_transform(train_data['ring-type'])

array([18, 39, 18, ..., 39, 27, 18])

In [ ]:
def encode_val(val):
    try :
        return float(val)
    except :
        return val

for x_col1 in train_data.drop(columns=['id']).columns:
    train_data[x_col1] = train_data[x_col1].apply(lambda x: encode_val(x))

for x_col2 in test_data.drop(columns=['id']).columns:
    test_data[x_col2] = test_data[x_col2].apply(lambda x: encode_val(x))

In [ ]:
train_data.columns

Index(['id', 'class', 'cap-diameter', 'cap-shape', 'cap-surface', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color',
       'stem-height', 'stem-width', 'stem-root', 'stem-surface', 'stem-color',
       'veil-type', 'veil-color', 'has-ring', 'ring-type', 'spore-print-color',
       'habitat', 'season'],
      dtype='object')

In [ ]:
print(train_data['cap-shape'].mode())
print(train_data['cap-shape'].mode()[0])

0    x
Name: cap-shape, dtype: object
x


In [ ]:
def fix_missing(data,col):
    if data[col].dtypes == 'float64' or data[col].dtypes == 'int64':
        return data[col].fillna(data[col].mean(),inplace=True)
    else:
        return data[col].fillna(data[col].mode()[0],inplace=True)

for x_col1 in train_data.drop(columns=['id','class']).columns:
    fix_missing(train_data,x_col1)

for x_col2 in test_data.drop(columns=['id']).columns:
    fix_missing(test_data,x_col2)

/tmp/ipykernel_33/3088943262.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  return data[col].fillna(data[col].mean(),inplace=True)
/tmp/ipykernel_33/3088943262.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

In [ ]:
train_data.isna().sum()

id                      0
class                   0
cap-diameter            0
cap-shape               0
cap-surface             0
cap-color               0
does-bruise-or-bleed    0
gill-attachment         0
gill-spacing            0
gill-color              0
stem-height             0
stem-width              0
stem-root               0
stem-surface            0
stem-color              0
veil-type               0
veil-color              0
has-ring                0
ring-type               0
spore-print-color       0
habitat                 0
season                  0
dtype: int64

In [ ]:
print(train_data['cap-diameter'].dtypes == 'float64')

True


In [ ]:
train_data.loc[0,'cap-diameter']

8.8

In [ ]:
# for j in range(len(train_data['veil-color'].values)):
#     if isinstance(train_data['veil-color'].values[j],int) or isinstance(train_data['veil-color'].values[j],float):
#         print(train_data['veil-color'].values[j])

In [ ]:
with np.nditer(train_data['veil-color'].values, flags=['refs_ok']) as it:
    for n1 in it:
        if isinstance(n1.item(), int) or isinstance(n1.item(), float):
            print(n1.item())

8.25
2.49
3.32


In [ ]:
def value_exchange():
    arr = train_data['veil-color'].values
    with np.nditer(arr, flags=['refs_ok']) as it:
        for n1 in it:
            if isinstance(n1.item(), int) or isinstance(n1.item(), float):
                print(n1.item())
value_exchange()

8.25
2.49
3.32


In [ ]:
def label_encode(data,col):
    vals = list(data[col].unique())
    arr = data[col].values
    k = 0.0
    d = {}
    for i in range(len(vals)):
        if not isinstance(vals[i],int) and not isinstance(vals[i],float):
            d[vals[i]] = k
            k+=1

    with np.nditer(arr, flags=['refs_ok']) as it:
        for j,n1 in enumerate(it):
            if not isinstance(n1.item(), int) and not isinstance(n1.item(), float):
                arr[j] = float(d[n1.item()])
            else:
                continue
    data[col] = pd.DataFrame(arr,columns=[col],dtype=np.float64)
    return data[col]

for x_col1 in train_data.drop(columns=['id','cap-diameter']).columns:
    train_data[x_col1] = label_encode(train_data,x_col1)

In [ ]:
for x_col2 in test_data.drop(columns=['id','cap-diameter']).columns:
    label_encode(test_data,x_col2)

In [ ]:
train_data.head()

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,0.0,8.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1.0,4.51,1.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
2,2,0.0,6.94,0.0,0.0,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,3,0.0,3.88,0.0,2.0,3.0,0.0,2.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,4,0.0,5.85,1.0,3.0,4.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0


In [ ]:
train_data['cap-shape']

0          0.0
1          1.0
2          0.0
3          0.0
4          1.0
          ... 
3116940    0.0
3116941    6.0
3116942    1.0
3116943    2.0
3116944    1.0
Name: cap-shape, Length: 3116945, dtype: float64

In [ ]:
type(train_data.loc[0,'cap-shape'])

numpy.float64

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(train_data.drop(columns=['id','class']),train_data['class'],test_size=0.3,random_state=0)

In [ ]:
X_train = X_train.to_numpy()
Y_train = Y_train.to_numpy()
X_test = X_test.to_numpy()
Y_test = Y_test.to_numpy()

In [ ]:
Y_test

array([1., 0., 0., ..., 1., 1., 0.])

In [ ]:
train_data.shape

(3116945, 22)

In [ ]:
X_train.shape

(2181861, 20)

In [ ]:
class MushroomClassifier(Model):
    def __init__(self):
        super().__init__()
#         self.layer1 = Dense(22,activation="relu")
#         self.batchNorm = BatchNormalization()
#         self.layer2 = Dense(22,activation="relu")
#         self.layer3 = Dense(22,activation="relu")
#         self.final_layer = Dense(1,activation="softmax")

        self.layer1 = Sequential([
                        Input(shape=(20,),dtype=np.float64),
                        Dense(20,activation='relu'),
                        Dense(10,activation='relu'),
                        Dense(10,activation='relu'),
                        Dense(2,activation='relu'),
                        Flatten(),
                        Dense(1,activation='sigmoid')])

    def call(self,x,training):
        return self.layer1(x)


# class MushroomModel(Model):
#     def __init__(self,model):
#         self.mode

m_model = MushroomClassifier()
print(m_model.summary())

Model: "mushroom_classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 1)              │           765 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 765 (2.99 KB)

 Trainable params: 765 (2.99 KB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
tf.constant(X_train)
tf.constant([[1,2,3],[4,5,6]])

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]], dtype=int32)>

In [ ]:
from tensorflow.keras import backend as k
from keras.src import ops
from keras.src.losses.loss import squeeze_or_expand_to_same_rank
class MCCLoss(Loss):
    def __init__(self):
        super().__init__()

    def call(self,y_true,y_pred):
#         y_pred = tf.round(tf.clip_by_value(y_pred, 0, 1))

        # Convert to float tensors
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)

        # Calculate the confusion matrix elements
        tp = tf.reduce_sum(tf.cast(y_true * y_pred, tf.float32))
        tn = tf.reduce_sum(tf.cast((1 - y_true) * (1 - y_pred), tf.float32))
        fp = tf.reduce_sum(tf.cast((1 - y_true) * y_pred, tf.float32))
        fn = tf.reduce_sum(tf.cast(y_true * (1 - y_pred), tf.float32))

        # Calculate MCC
        numerator = (tp * tn) - (fp * fn)
        denominator = tf.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

        mcc = tf.where(tf.equal(denominator, 0), 0.0, numerator / denominator)

        return mcc

In [ ]:
m_model.compile(loss=MCCLoss(),optimizer=SGD(),metrics=["acc"])

In [ ]:
Y_train

array([1., 1., 0., ..., 1., 1., 0.])

In [ ]:
m_model.fit(tf.constant(X_train),tf.constant(Y_train),verbose=1,epochs=10)

Epoch 1/10
68184/68184 ━━━━━━━━━━━━━━━━━━━━ 97s 1ms/step - acc: 0.4610 - loss: 6.8800e-11
Epoch 2/10
25332/68184 ━━━━━━━━━━━━━━━━━━━━ 1:01 1ms/step - acc: 0.4607 - loss: 9.4282e-11

In [ ]:
preds = m_model.predict(test_data.drop(columns='id').to_numpy())

In [ ]:
preds

In [ ]:
# with np.nditer(arr, flags=['refs_ok']) as it:
#     for

In [ ]:
# final_output = pd.concat([pd.DataFrame(test_data['id']),pd.DataFrame(preds)],axis=1,ignore_index=True)